# Map-making with Python
### By Maggie Orton and Alex Cao
### April 19, 2017
### <a href="http://cscar.research.umich.edu">CSCAR</a> at The University of Michigan  

Please fill out the workshop sign-in <a href="">here</a>

This workshop will cover how to make a few basic types of maps using matplotlib in Python. The types of maps covered are:
1. Choropleth
2. Dot map
3. Proportional dot map
4. Isopleth

#### Note:  
to install geopandas on a Mac with Anaconda:  
$ conda install -c https://conda.anaconda.org/ioos geopandas

In [1]:
#% matplotlib inline

# Choropleth

Importing required modules and limiting pandas output to 12 rows:

In [2]:
import geopandas as gp
import pandas as pd
import time
import requests
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.collections import PolyCollection

pd.options.display.max_rows = 12

### Generating GeoDataFrame of MI counties

In [3]:
# Read GeoJSON from data portal and convert to GeoDataFrame
R = requests.get(r'http://gis-michigan.opendata.arcgis.com/datasets/67a8ff23b5f54f15b7133b8c30981441_0.geojson')
R.raise_for_status()
response = R.json()
counties = gp.GeoDataFrame.from_features(response['features'])
# Drop unnecessary columns
counties.drop(['ACRES','CNTY_CODE','FIPSCODE','LABEL','SQKM','ShapeSTArea','ShapeSTLength','TYPE','VER'], axis=1, inplace=True)
# Find geographic center of counties
counties['centroid'] = counties.geometry.apply(lambda x: x.centroid)

### Loading choropleth snow crash data

In [4]:
# Load choropleth data
fatals = pd.read_csv('snow_crashes.csv', sep='\t')
fatals.set_index('County', inplace=True)
# convert string to integers
fatals = fatals.applymap(lambda x: int(str(x).replace(',','')))
fatals['snow_crashes'] = fatals['Total']

# merge snow crash data with geodataframe
df = counties.merge(pd.DataFrame(fatals['snow_crashes']), how='inner', left_on='NAME', right_index=True)
df.head(2)

,FIPSNUM,LAYOUT,NAME,OBJECTID,PENINSULA,SQMILES,geometry,centroid,snow_crashes
0,1,landscape,Alcona,1,lower,694.424,POLYGON ((-83.88711741272411 44.55385437631399...,POINT (-83.5937944829822 44.68542260456106),21
1,3,landscape,Alger,2,upper,936.247,(POLYGON ((-87.11602141361914 46.2772599477443...,POINT (-86.60393525888711 46.40859154770666),37


### Formatting colors and labels

In [5]:
# function assigns x a color based on minimums set in bins (inclusive)
# requires: same number of bins and colors
def assign_color(x, colors, bins):
    bincount = len(bins)
    counter = 1
    while (counter < bincount):
        if x >= bins[-counter]:
            return colors[-counter]
        counter = counter + 1
    return colors[0]

In [6]:
# example: assigning '5' to blue in 5-9 bin
assign_color(5, ['green', 'blue', 'red'], [0, 5, 10])

'blue'

In [7]:
# setting up choropleth parameters
colors = ['#edf8e9','#bae4b3','#74c476','#31a354','#006d2c']
bins = [0,100,200,400,800]
labels = []
# formatting labels for bins
for i in range(len(bins)):
    if i < len(bins)-1:
        label = '{} - {}'.format(bins[i]+1,bins[i+1])
    else:
        label = '{}+'.format(bins[i])
    labels.append(label)
labels

['1 - 100', '101 - 200', '201 - 400', '401 - 800', '800+']

In [8]:
df['color'] = df['snow_crashes'].apply(assign_color, args=(colors,bins))
df[['snow_crashes', 'color']].head(4)

,snow_crashes,color
0,21,#edf8e9
1,37,#edf8e9
2,429,#31a354
3,52,#edf8e9


### Plotting choropleth

In [9]:
inches = 12 # Approximately 1200 x 1200 pixels
fig = plt.figure(2, figsize=(inches,inches), frameon=True)
ax = plt.gca()
# add blue for the great lakes (and land)
fig.patch.set_facecolor('#9bbff4')

In [10]:
# create a nested list of polygons, vertices, and xy-coordinates 
mi = []
for i in range(df.shape[0]):
    geom = counties.at[i,'geometry']
    if geom.geom_type == 'Polygon':
        poly_xy = [list(n) for n in geom.exterior.coords]
        mi.append(poly_xy)
    elif geom.geom_type == 'MultiPolygon':
        for poly in geom:
            poly_xy = [list(n) for n in poly.exterior.coords]
            mi.append(poly_xy)

In [11]:
# Make a polygon collection and plot it with colour column
coll = PolyCollection(mi, facecolors=df['color'], edgecolors='white')
ax.add_collection(coll)
# this line is necessary for collection to prevent crashing
ax.autoscale_view()

In [12]:
# Plot county text
fs = 7
for i in range(df.shape[0]):
    xy = list(df.at[i,'centroid'].coords)[0]
    xc = xy[0]
    yc = xy[1]
    name = df.at[i,'NAME']
    plt.text(xc, yc, name, fontsize=fs, ha='center', weight='bold')        

In [13]:
# Draw legend
alpha = .8
swatches = []
for color, label in zip(colors,labels):
    swatches.append(mpatches.Patch(color=color, label=label, alpha=alpha))
plt.legend(handles=swatches, bbox_to_anchor=(0.4,0.3), frameon=False)

In [14]:
# Plot title
plt.figtext(0.33,0.32, 'Vehicle Crashes\nIn Snowy Conditions', fontsize=fs+3, weight='bold', ha='center', va='bottom')

In [15]:
# Format figure and save
plt.axis('off')
plt.tight_layout()
fig.savefig("mi_choropleth.png", facecolor=fig.get_facecolor(), bbox_inches='tight')
plt.gcf().clear()

/Users/maggieorton/anaconda/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


# Dot Map

### Creating new map to add dots to

In [81]:
fig2 = plt.figure(2, figsize=(inches,inches), frameon=True)
ax2 = plt.gca()
# add blue for the great lakes (and land)
fig2.patch.set_facecolor('#9bbff4')

In [82]:
# Make a polygon collection and plot it
coll = PolyCollection(mi, facecolors='#edf8e9', edgecolors='white')
ax2.add_collection(coll)
# this line is necessary for collection to prevent crashing
ax2.autoscale_view()

In [83]:
# Plot title
plt.figtext(0.5,0.9, 'Snowmobile Crashes', fontsize=fs+20, weight='bold', ha='center', va='bottom')

### Reading in snowmobile crash data

In [84]:
snowmobiles = pd.read_table("snowmobile_crashes.txt", sep = "\t", usecols = ["Crash Longitude", "Crash Latitude"])
snowmobiles[:3]

,Crash Longitude,Crash Latitude
0,-85.800058,46.180844
1,-84.789822,45.440444
2,-86.740536,46.321993


In [85]:
snowmobiles.applymap(lambda x: float(str(x).replace(' ','')))
# remove rows from df with long/lat = 0
snowmobiles.drop(snowmobiles[snowmobiles['Crash Longitude'] == 0].index, inplace = True)
snowmobiles

,Crash Longitude,Crash Latitude
0,-85.800058,46.180844
1,-84.789822,45.440444
2,-86.740536,46.321993
3,-85.860376,42.738967
4,-85.836189,44.772364
5,-85.483634,41.875492
...,...,...
113,-83.198362,42.392715
114,-83.202282,42.340796
115,-82.557579,42.948126


### Plotting circles

In [86]:
#plt.plot(snowmobiles['Crash Longitude'], snowmobiles['Crash Latitude'], 'wo')
for lat, lon, city in zip(snowmobiles['Crash Latitude'], snowmobiles['Crash Longitude']):
    circ = plt.Circle((lon, lat), radius = .05, color = 'maroon', alpha = .6, fill = True) #, label = "crash")
    ax2.add_artist(circ)

In [87]:
# Format figure and save
plt.axis('off')
plt.tight_layout()
fig2.savefig("snowmobile_crashes.png", facecolor=fig2.get_facecolor(), bbox_inches='tight')
plt.gcf().clear()

# Proportional Dot Map

### Creating new map to add dots to

In [88]:
fig3 = plt.figure(2, figsize=(inches,inches), frameon=True)
ax3 = plt.gca()
# add blue for the great lakes (and land)
fig3.patch.set_facecolor('#9bbff4')

In [89]:
# Make a polygon collection and plot it
coll = PolyCollection(mi, facecolors='#edf8e9', edgecolors='white')
ax3.add_collection(coll)
# this line is necessary for collection to prevent crashing
ax3.autoscale_view()

In [90]:
# Plot title
plt.figtext(0.5, .9, 'Top 20 Deer Crashes', fontsize=fs+20, weight='bold', ha='center', va='bottom')

### Reading in deer crash data

In [91]:
deer = pd.read_csv("deer_in_the_city.txt", usecols = ["city", "Total","Lat","Lon"]).sort_values(by = ["Total"])
# take top 20
deer = deer[-20:]
deer

,city,Total,Lat,Lon
19,Burton,40,42.999472,-83.616342
18,Norton Shores,41,43.168904,-86.263946
17,Sterling Heights,44,42.580312,-83.030203
16,Southfield,44,42.473369,-83.221873
15,Wyoming,46,42.913360,-85.705309
14,East Lansing,55,42.736979,-84.483865
...,...,...,...,...
5,Ann Arbor,90,42.280826,-83.743038
4,Farmington Hills,95,42.498994,-83.367717
3,Battle Creek,116,42.321152,-85.179714


In [92]:
deer["Total"] = pd.to_numeric(deer["Total"])

### Plotting circles

In [93]:
#plt.plot(deer['Longitude'], deer['Latitude'], 'wo')
for lat, lon, num in zip(deer['Lat'], deer['Lon'], deer['Total']):
    circ = plt.Circle((lon, lat), radius = num / 500, color = 'maroon', alpha = .6, fill = True)
    ax3.add_artist(circ)

In [94]:
# Format figure and save
plt.axis('off')
plt.tight_layout()
fig3.savefig("top_20_crashes.png", facecolor=fig2.get_facecolor(), bbox_inches='tight')
plt.gcf().clear()

# Isopleth

In [30]:
import matplotlib
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

temps = pd.read_csv("mi_temperature.txt", usecols = ["latitude", "longitude", "maxTemp"])
X, Y = np.meshgrid(temps["latitude"], temps["longitude"])
Z = temps["maxTemp"]
#fig4 = plt.figure()
print(X)
print(Y)
print(Z)
"""
CS = plt.contour(X, Y, Z)
plt.clabel(CS, inline = 1, fontsize = 10)
plt.axis('off')
plt.tight_layout()
plt.title('Isopleth')
fig4.savefig("isopleth.png")
"""


"""
# Create a simple contour plot with labels using default colors.  The
# inline argument to clabel will control whether the labels are draw
# over the line segments of the contour, removing the lines beneath
# the label
fig4 = plt.figure()
CS = plt.contour(X, Y, Z)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Simplest default with labels')
fig4.savefig("contour.png")
"""

[[ 46.6  46.5  46.6 ...,  42.9  43.   43.1]
 [ 46.6  46.5  46.6 ...,  42.9  43.   43.1]
 [ 46.6  46.5  46.6 ...,  42.9  43.   43.1]
 ..., 
 [ 46.6  46.5  46.6 ...,  42.9  43.   43.1]
 [ 46.6  46.5  46.6 ...,  42.9  43.   43.1]
 [ 46.6  46.5  46.6 ...,  42.9  43.   43.1]]
[[-90.3 -90.3 -90.3 ..., -90.3 -90.3 -90.3]
 [-90.2 -90.2 -90.2 ..., -90.2 -90.2 -90.2]
 [-90.2 -90.2 -90.2 ..., -90.2 -90.2 -90.2]
 ..., 
 [-82.5 -82.5 -82.5 ..., -82.5 -82.5 -82.5]
 [-82.5 -82.5 -82.5 ..., -82.5 -82.5 -82.5]
 [-82.5 -82.5 -82.5 ..., -82.5 -82.5 -82.5]]
0       28.55
1       28.45
2       28.26
3       29.20
4       28.22
5       28.46
        ...  
1700    44.72
1701    50.80
1702    50.34
1703    50.02
1704    49.91
1705    49.47
Name: maxTemp, dtype: float64


'\n# Create a simple contour plot with labels using default colors.  The\n# inline argument to clabel will control whether the labels are draw\n# over the line segments of the contour, removing the lines beneath\n# the label\nfig4 = plt.figure()\nCS = plt.contour(X, Y, Z)\nplt.clabel(CS, inline=1, fontsize=10)\nplt.title(\'Simplest default with labels\')\nfig4.savefig("contour.png")\n'

In [31]:
Z1 = mlab.bivariate_normal(X, Y, 1.0, 1.0, 0.0, 0.0)
Z2 = mlab.bivariate_normal(X, Y, 1.5, 0.5, 1, 1)
# difference of Gaussians
Z = 10.0 * (Z2 - Z1)
print(Z1)
print(Z2)
print(Z)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
